In [ ]:
# 1) 설치
!pip install transformers datasets accelerate --quiet

# 2) 로딩
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer, DataCollatorForLanguageModeling)
from datasets import Dataset

model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # 별도 임베딩 리사이즈 방지
model = AutoModelForCausalLM.from_pretrained(model_name)

# 3) 초소형 데이터
examples = [
  {"instruction": "안녕?", "input": "", "output": "안녕하세요! 만나서 반가워요."},
  {"instruction": "이름이 뭐야?", "input": "", "output": "저는 인공지능 챗봇이에요."},
  {"instruction": "날씨 어때?", "input": "", "output": "오늘은 맑고 따뜻한 날씨예요."},
]
ds = Dataset.from_list(examples)

# 4) 전처리: 프롬프트+라벨 구성 단순화
def to_sample(e):
    prompt = f"질문: {e['instruction']} {e['input']}\n답변:"
    target = e["output"]
    text = prompt + " " + target
    enc = tokenizer(text, truncation=True, max_length=64)
    # 패딩 토큰은 손실에서 제외(-100)
    enc["labels"] = enc["input_ids"].copy()
    return enc

tokenized = ds.map(to_sample, remove_columns=ds.column_names)

# 5) 데이터콜레이터 (Causal LM용)
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 6) 초경량 학습 설정: 스텝 제한 + 저장/로깅 최소화
training_args = TrainingArguments(
    output_dir="./kogpt2-ultralight",
    per_device_train_batch_size=1,
    num_train_epochs=30,            # 에폭은 1로
    max_steps=10,                  # 10 스텝만
    learning_rate=5e-5,
    save_strategy="no",            # 중간 저장 없음
    logging_steps=10,
    report_to="none",
    fp16=True,                     # GPU면 자동 혼합정밀 (지원 안되면 무시)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=collator,
)

trainer.train()

# 7) 저장 & 간단 챗봇
trainer.save_model("./kogpt2-ultralight")
tokenizer.save_pretrained("./kogpt2-ultralight")

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
tok = AutoTokenizer.from_pretrained("./kogpt2-ultralight")
mdl = AutoModelForCausalLM.from_pretrained("./kogpt2-ultralight")
chatbot = pipeline("text-generation", model=mdl, tokenizer=tok)

print("초경량 챗봇 시작 (종료 입력 시 끝)\n")
while True:
    q = input("사용자: ")
    if q.strip() == "종료": break
    prompt = f"질문: {q}\n답변:"
    out = chatbot(prompt, max_new_tokens=60, do_sample=True, temperature=0.8)[0]["generated_text"]
    ans = out.split("답변:")[-1].strip()
    print("🤖:", ans)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,1.708300


Device set to use cuda:0


초경량 챗봇 시작 (종료 입력 시 끝)

사용자: 안녕
🤖: 오늘은 맑고 따뜻한 날씨예요. 저는 오늘은
사용자: 날씨 어때
🤖: 오늘은 맑고 따뜻한 날씨예요. 어때. 지금은 맑고 따뜻한 날씨예요. : 오늘은 맑고 따뜻한 날씨예요. 저는 날씨 어때. 날씨 어때. 오늘은 맑고 따뜻한 날씨예요. 최근 들어 날씨가 따뜻해지고 있습니다.
따뜻한 날씨예요. 가을 날씨예요.
사용자: 이름이 뭐야
🤖: 인공지능 챗봇이에요. 챗봇이에
